# IMPORT

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, random_split
from torch import nn
from torch.optim.lr_scheduler import StepLR
import torch
from transformers import AutoTokenizer
import natasha
import mlflow
import json
from IPython.display import clear_output
from sklearn.preprocessing import LabelEncoder
import re
import psycopg2
from torchinfo import summary
from datetime import datetime
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")
DB_USER = os.environ.get("DB_USER")
DB_PASS = os.environ.get("DB_PASS")
DB_NAME = os.environ.get("DB_NAME")

In [3]:
current_date = datetime.strptime("2024-01-09 00:00:00", "%Y-%d-%m %H:%M:%S")

# FUNC

In [ ]:
def train_model(model_config, device, TRAIN_LOADER, TEST_LOADER,
                model = None, optimizer = None, scheduler = None,
                weight_decay = 1e-3,
                lr = 1e-3,
                start_epoch = 0,
                epochs = 10):
    try:
        if model == None:
            model = Forecaster(model_config).to(device)
        if optimizer == None:
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
        if scheduler is not None:
            scheduler = StepLR(optimizer, step_size=scheduler["step_size"], gamma=scheduler["gamma"])
        with open("artifacts/model_summary.txt", "w", encoding="utf-8") as f:
            f.write(str(summary(model)))
        mlflow.log_artifacts("artifacts")
        BCE = nn.BCELoss()
        for ep in tqdm(range(start_epoch, start_epoch + epochs)):
            epoch_loss_train = 0; epoch_loss_test = 0
            accuracy_train = 0; accuracy_test = 0
            # TRAIN
            for sample in tqdm(TRAIN_LOADER, desc = f"Epoch: {ep}"):
                X_time_ = sample[0].to(device); Add_feature_ts_ = sample[1].to(device); ticker=sample[2].to(device); Y_ = sample[3].to(device)
                pred = model(
                    time = X_time_,
                    add_time = Add_feature_ts_,
                    ticker=ticker
                ).reshape([-1])
                loss_train = BCE(pred, Y_.type(torch.float))
                optimizer.zero_grad()
                loss_train.backward()
                optimizer.step()
                epoch_loss_train += loss_train.item() * len(pred)
                accuracy_train += pred.round().int().eq(Y_).sum().item()
            # TEST
            for sample in tqdm(TEST_LOADER, desc = f"Epoch: {ep}"):
                X_time_ = sample[0].to(device); Add_feature_ts_ = sample[1].to(device); ticker=sample[2].to(device); Y_ = sample[3].to(device)
                with torch.no_grad():
                    pred = model(
                        time = X_time_,
                        add_time = Add_feature_ts_,
                        ticker=ticker
                    ).reshape([-1])
                loss_test = BCE(pred, Y_.type(torch.float))
                epoch_loss_test += loss_test.item() * len(pred)
                accuracy_test += pred.round().int().eq(Y_).sum().item()
            if scheduler is not None:
                scheduler.step()
            accuracy_train /= len(TRAIN_LOADER.dataset); accuracy_test /= len(TEST_LOADER.dataset)
            mlflow.log_metrics({"train loss":epoch_loss_train, "test loss":epoch_loss_test, "train_acc":accuracy_train, "test_acc":accuracy_test}, step=ep)
            if ep % 10 == 0:
                clear_output()
    except Exception as e:
        print(e)
    except KeyboardInterrupt:
        torch.save(model, "model.pt")
        torch.save(optimizer.state_dict(), "optimizer_state_dict.pt")
        print("save with error")
        return model, optimizer
    finally:
        torch.save(model, "model.pt")
        torch.save(optimizer.state_dict(), "optimizer_state_dict.pt")
        print("done")
        return model, optimizer

In [5]:
def pd_to_np_time_seties(df_pd:pd.DataFrame, index_column_time=None, x_step=50, y_step=10, columns_x=[1, 2, 3, 4, 5], columns_y=[2, 3]) -> np.array:   
    df_pd = df_pd.reset_index(drop=True)
    if index_column_time != None:
       time = df_pd.loc[:, index_column_time].copy()
    
    num_samples = df_pd.shape[0]
    num_samples = np.floor(num_samples / (x_step+y_step)).astype(int)
    y_features = len(columns_y)
    x_features = len(columns_x)

    

    X, Y = np.empty((num_samples, x_step, x_features)), np.empty((num_samples, y_step, y_features))
    if index_column_time != None:
        T = np.empty((num_samples, 2), dtype=pd.Timestamp)

    start_x = 0
    start_y = x_step
    for index in tqdm(range(0, num_samples)):
        if df_pd.loc[start_x:start_x + x_step - 1, columns_x].shape[0] == 0:
            print(df_pd.loc[start_x-60:start_x + x_step - 1, columns_x])
            print('start, end', start_x, start_y)
        X[index] = df_pd.loc[start_x:start_x + x_step - 1, columns_x].to_numpy().reshape((x_step, x_features))
        
        Y[index] = df_pd.loc[start_y:start_y + y_step - 1, columns_y].to_numpy().reshape((y_step, y_features))
        
        if index_column_time != None:
            T[index] = [time.loc[start_x:start_x + x_step].min(), time.loc[start_x:start_x + x_step].max()]

        start_x = start_x + x_step + y_step
        start_y = start_y + x_step + y_step
        
    if index_column_time != None:
        return X, Y, T
    else:
        return X, Y

In [6]:
def compute_rsi(prices: torch.Tensor, period: int = 14):
    """
    Вычисляет RSI для каждого временного ряда из батча.
    prices: тензор размера [batch, seq_len]
    Возвращает тензор RSI для последнего временного шага: [batch]
    """
    # Вычисляем изменения цены
    delta = prices[:, 1:] - prices[:, :-1]  # [batch, seq_len-1]
    # Вычисляем прирост и потери
    gain = torch.clamp(delta, min=0)
    loss = torch.clamp(-delta, min=0)
    
    # Если шагов меньше, чем период, возвращаем NaN
    if delta.shape[1] < period:
        raise ValueError("Длина временного ряда меньше, чем период RSI")
    
    # Усредняем приросты и потери по последнему периоду
    avg_gain = gain[:, -period:].mean(dim=1)
    avg_loss = loss[:, -period:].mean(dim=1)
    
    rs = avg_gain / (avg_loss + 1e-8)  # добавляем маленькое число, чтобы не делить на 0
    rsi = 100 - (100 / (1 + rs))
    return rsi  # [batch]

def compute_ema(prices: torch.Tensor, period: int):
    """
    Вычисляет EMA по заданному периоду для каждого временного ряда.
    prices: тензор размера [batch, seq_len]
    Возвращает EMA с той же размерностью [batch, seq_len].
    """
    alpha = 2 / (period + 1)
    ema = torch.zeros_like(prices)
    ema[:, 0] = prices[:, 0]  # инициализация первым значением
    # Итеративное вычисление EMA
    for t in range(1, prices.shape[1]):
        ema[:, t] = alpha * prices[:, t] + (1 - alpha) * ema[:, t-1]
    return ema

def compute_macd(prices: torch.Tensor, short_period: int = 5, long_period: int = 12, signal_period: int = 5):
    """
    Вычисляет MACD, сигнальную линию и гистограмму для каждого временного ряда.
    Из-за малого числа шагов в ряду (24 шага) стандартные периоды (12, 26, 9) могут быть не применимы.
    Поэтому можно использовать меньшие периоды.
    """
    ema_short = compute_ema(prices, short_period)
    ema_long  = compute_ema(prices, long_period)
    
    # MACD как разница EMA короткого и длинного периодов
    macd = ema_short - ema_long
    
    # Сигнальная линия – EMA от MACD
    signal_line = compute_ema(macd, signal_period)
    # Гистограмма – разница между MACD и сигнальной линией
    histogram = macd - signal_line
    return macd, signal_line, histogram

# TS

In [7]:
query = f"""SELECT * FROM price_hour"""
with psycopg2.connect(dbname = DB_NAME, user = DB_USER, password = DB_PASS, host = DB_HOST, port = DB_PORT) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        time_series = pd.DataFrame(cur.fetchall(), columns=["open", "low", "high", "close", "volume", "date", "ticker"])

In [8]:
# optional
time_series = time_series[time_series["date"] <= current_date]
tickers_ = np.array(['MGNT', 'PHOR', 'MAGN', 'VTBR', 'RUAL', 'ALRS', 'FLOT', 'TATN',
       'RTKM', 'MTSS', 'NVTK', 'SNGS', 'ROSN', 'CHMF', 'KMAZ', 'YDEX',
       'AFLT', 'SBER', 'MVID', 'MTLR', 'GAZP', 'LENT', 'BSPB', 'SIBN',
       'HYDR', 'NLMK', 'RASP', 'GMKN', 'AFKS', 'PLZL', 'LKOH'],
      dtype=object)


time_series = time_series[time_series["ticker"].isin(tickers_)]
time_series.reset_index(inplace=True, drop=True)

In [9]:
time_series['weekday'] = time_series['date'].apply(lambda row: row.weekday()) + 1
time_series['hour'] = time_series['date'].apply(lambda row: row.hour)
time_series['hour_sin'] = np.sin(np.pi * 2 * time_series['hour'] / 24)
time_series['hour_cos'] = np.cos(np.pi * 2 * time_series['hour'] / 24)
time_series['weekday_sin'] = np.sin(np.pi * 2 * time_series['weekday'] / 7)
time_series['weekday_cos'] = np.cos(np.pi * 2 * time_series['weekday'] / 7)

In [10]:
time_series

,open,low,high,close,volume,date,ticker,weekday,hour,hour_sin,hour_cos,weekday_sin,weekday_cos
0,5601.0,5601.0,5601.0,5601.0,13,2021-08-30 09:00:00,MGNT,1,9,7.071068e-01,-0.707107,0.781831,0.62349
1,5601.0,5578.5,5623.5,5590.0,38026,2021-08-30 10:00:00,MGNT,1,10,5.000000e-01,-0.866025,0.781831,0.62349
2,5592.0,5558.0,5593.5,5577.5,26215,2021-08-30 11:00:00,MGNT,1,11,2.588190e-01,-0.965926,0.781831,0.62349
3,5577.5,5557.5,5577.5,5561.5,14164,2021-08-30 12:00:00,MGNT,1,12,1.224647e-16,-1.000000,0.781831,0.62349
4,5561.5,5544.0,5579.0,5560.0,18672,2021-08-30 13:00:00,MGNT,1,13,-2.588190e-01,-0.965926,0.781831,0.62349
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380727,6131.0,6116.0,6132.0,6132.0,400,2024-08-31 19:00:00,LKOH,6,19,-9.659258e-01,0.258819,-0.781831,0.62349
380728,6132.0,6124.0,6139.5,6133.0,281,2024-08-31 20:00:00,LKOH,6,20,-8.660254e-01,0.500000,-0.781831,0.62349
380729,6133.0,6112.5,6144.0,6120.0,719,2024-08-31 21:00:00,LKOH,6,21,-7.071068e-01,0.707107,-0.781831,0.62349
380730,6120.0,6119.0,6133.5,6122.5,245,2024-08-31 22:00:00,LKOH,6,22,-5.000000e-01,0.866025,-0.781831,0.62349


In [11]:
x_step = 24
y_step = 12
x_column = ['open', 'low', 'high', 'close', 'volume', 'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']
y_column = ['close']
x_feature = len(x_column)
y_feature = len(y_column)

In [12]:
def return_batch(time_series):
    X_time = np.zeros((0, x_step, x_feature))
    Y = np.zeros((0, y_step, y_feature))
    T = np.zeros((0, 2))
    TICK = np.empty((0))
    for ticker in time_series['ticker'].unique():
        X_time_, Y_, T_ = pd_to_np_time_seties(time_series[time_series['ticker'] == ticker], 
                                            index_column_time='date', 
                                            x_step=x_step, 
                                            y_step=y_step, 
                                            columns_x=x_column, 
                                            columns_y=y_column)
        
        X_time = np.concatenate((X_time, X_time_), axis=0)
        Y = np.concatenate((Y, Y_), axis=0)
        T = np.concatenate((T, T_), axis=0)
        TICK = np.concatenate((TICK, np.array([ticker] * X_time_.shape[0])), axis=0)

    X_time = torch.from_numpy(X_time).float()
    Y = torch.from_numpy(Y).float()
    if Y.shape[-1] == 1:
        Y = Y.reshape([Y.shape[0], -1])

    Y = ((Y - X_time[:, -1:, 3]) / X_time[:, -1:, 3]) > 0.01
    Y = Y.sum(dim=1)
    Y[Y !=0 ] = 1

    X_time[:, :, :5] = torch.log10(X_time[:, :, :5])
    X_time[X_time == -torch.inf] = 0

    Add_feature_ts = torch.empty([X_time.shape[0], 5])
    # rsi
    Add_feature_ts[:, 0] = compute_rsi(X_time[:, :, 3])
    # mean
    Add_feature_ts[:, 1] = X_time[:, :, 3].mean(dim=1)
    Add_feature_ts[:, 2] = X_time[:, :, 4].mean(dim=1)
    # std
    Add_feature_ts[:, 3] = X_time[:, :, 3].std(dim=1)
    Add_feature_ts[:, 4] = X_time[:, :, 4].std(dim=1)

    # macd, signal_line, histogram
    macd, signal_line, histogram = compute_macd(X_time[:, :, 3], 3, 5, 4)
    X_time = torch.cat([X_time, macd.unsqueeze(-1), signal_line.unsqueeze(-1), histogram.unsqueeze(-1)], dim=2)

    return X_time, Add_feature_ts, TICK, Y

In [13]:
unique_date = np.sort(time_series["date"].unique())
sep_index = int(len(unique_date) * 0.85)

In [14]:
X_time_train, Add_feature_ts_train, TICK_train, Y_train = return_batch(time_series[time_series["date"].isin(unique_date[:sep_index])])
X_time_test, Add_feature_ts_test, TICK_test, Y_test = return_batch(time_series[time_series["date"].isin(unique_date[sep_index:])])

100%|██████████| 54/54 [00:00<00:00, 1255.84it/s]


In [15]:
label_encoder = LabelEncoder().fit(tickers_)
TICK_train = torch.from_numpy(label_encoder.transform(TICK_train)).int()
TICK_test = torch.from_numpy(label_encoder.transform(TICK_test)).int()

In [16]:
Y_train.unique(return_counts=True)

(tensor([0, 1]), tensor([5899, 3017]))

In [17]:
Y_test.unique(return_counts=True)

(tensor([0, 1]), tensor([1225,  400]))

In [18]:
def class_balance(Y):
    indices_class_0 = torch.nonzero(Y == 0).squeeze()
    indices_class_1 = torch.nonzero(Y == 1).squeeze()

    class_sample = Y.unique(return_counts=True)[1].min().item()

    torch.manual_seed(12312)
    permuted_indices_class_0 = indices_class_0[torch.randperm(len(indices_class_0))][:class_sample]
    torch.manual_seed(12312)
    permuted_indices_class_1 = indices_class_1[torch.randperm(len(indices_class_1))][:class_sample]
    permuted_indices = torch.cat([permuted_indices_class_0, permuted_indices_class_1])
    return permuted_indices

In [19]:
train_class_balance = class_balance(Y_train)
test_class_balance = class_balance(Y_test)
X_time_train, Add_feature_ts_train, TICK_train, Y_train = X_time_train[train_class_balance], Add_feature_ts_train[train_class_balance], TICK_train[train_class_balance], Y_train[train_class_balance]
X_time_test, Add_feature_ts_test, TICK_test, Y_test = X_time_test[test_class_balance], Add_feature_ts_test[test_class_balance], TICK_test[test_class_balance], Y_test[test_class_balance]

In [20]:
# TRAIN_LOADER
TRAIN_LOADER = TensorDataset(X_time_train, Add_feature_ts_train, TICK_train, Y_train)
TEST_LOADER = TensorDataset(X_time_test, Add_feature_ts_test, TICK_test, Y_test)
batch_size = 50
TRAIN_LOADER = DataLoader(TRAIN_LOADER, batch_size=batch_size)
TEST_LOADER = DataLoader(TEST_LOADER, batch_size=batch_size)

# MODEL

In [24]:
name_experiment = "LSTM_class"

class Forecaster(nn.Module):
    def __init__(self, config_, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.config = {
            "x_feature" : 4,
            "hidden_size" : 36,
            "num_layers" : 4,
            "n_add_feature_ts" : 5,
            "y_feature": 1,
            "n_ticker":0,
            "ticker_embedd_dim":0
        }
        if config_ != None:
            for key in config_.keys():
                self.config[key] = config_[key]
        if config_["n_ticker"] != 0:
            self.ticker_embedd = nn.Embedding(config_["n_ticker"], config_["ticker_embedd_dim"])
        self.lstm = nn.LSTM(self.config["x_feature"], self.config["hidden_size"], self.config["num_layers"], batch_first=True)
        n_feature = self.config["hidden_size"] + self.config["n_add_feature_ts"] + config_["ticker_embedd_dim"]
        self.fc = nn.Sequential(
            nn.LayerNorm(n_feature),
            nn.SiLU(),
            nn.Linear(n_feature, self.config["y_feature"])
        )

    def forward(self, time:torch.Tensor, add_time:torch.Tensor=None, ticker:torch.Tensor=None, return_logit:bool=False):
        h0 = torch.zeros(self.config["num_layers"], time.size(0), self.config["hidden_size"]).to(time.device)
        c0 = torch.zeros(self.config["num_layers"], time.size(0), self.config["hidden_size"]).to(time.device)
        out, _ = self.lstm(time, (h0, c0))
        out = out[:, -1, :]
        if self.config["n_add_feature_ts"] != 0:
            out = torch.cat([out, add_time], dim=1)
        if self.config["n_ticker"] != 0:
            ticker_embedding = self.ticker_embedd(ticker)
            out = torch.cat([out, ticker_embedding], dim=1)
        out = self.fc(out)
        if return_logit:
            return out
        else:
            return torch.nn.functional.sigmoid(out)

# TRAIN

In [ ]:
# mlflow.set_experiment(name_experiment)

<Experiment: artifact_location='file:///c:/Users/Stanislav/Documents/py/Data%20Science%20%28DS%29/dipom_pet/mlruns/181443399550379659', creation_time=1742792711462, experiment_id='181443399550379659', last_update_time=1742792711462, lifecycle_stage='active', name='LSTM_class', tags={}>

In [ ]:
model_config = {
        "x_feature" : X_time_train.shape[-1],
        "hidden_size" : 36,
        "num_layers" : 4,
        "n_add_feature_ts" : Add_feature_ts_train.shape[-1],
        "y_feature": 1,
        "n_ticker":len(tickers_),
        "ticker_embedd_dim":4
        }
# model_config= {
#         "attention_layer" : 12,
#         "hidden_size" : 36,
#         "num_heads" : 4,
#         "x_feature": X_time.shape[-1],
#         "x_step": X_time.shape[1],
#         "y_step": y_step,
#         }

scheduler = {
    "step_size":100,
    "gamma":0.1
}

In [ ]:
import shutil

In [ ]:
# Первый запуск
try:
    run = mlflow.start_run()
    shutil.rmtree("artifacts/", ignore_errors=True)
    os.makedirs("artifacts/", exist_ok=True)
    with open("artifacts/model_config.json", "w") as f:
        json.dump(model_config, f)
    model, optimizer = train_model(model_config, device, TRAIN_LOADER, TEST_LOADER,
                                   epochs=1500, lr=1e-5, weight_decay=0)
finally:
    mlflow.end_run()

100%|██████████| 1500/1500 [14:06<00:00,  1.77it/s]

done
